In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_article_scd 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) AS
WITH tmp AS (
    SELECT 
        *,
        LAG(date_to) OVER (PARTITION BY assortment, article, last_update_date ORDER BY date_to DESC) date_to2,
        LAG(date_from) OVER (PARTITION BY assortment, article, last_update_date ORDER BY date_to DESC) date_from2,
        FIRST_VALUE(date_to) OVER (PARTITION BY assortment, article, last_update_date ORDER BY date_to DESC) date_to_latest,
        FIRST_VALUE(date_from) OVER (PARTITION BY assortment, article, last_update_date ORDER BY date_to DESC) date_from_latest
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_asm_article a
),
tmp2 AS (
    SELECT 
        DISTINCT assortment, article, is_delete, lifnr AS vendor_id,
        CASE WHEN date_to > date_from_latest AND date_to < date_to_latest THEN LEAST(date_from,date_from_latest) ELSE date_from END date_from,
        CASE WHEN date_to > date_from_latest AND date_to < date_to_latest THEN date_to_latest ELSE date_to END date_to,
        last_update_date
    FROM tmp
),
tmp3 AS (
    SELECT assortment, article, last_update_date, date_to, is_delete, vendor_id, MIN(date_from) date_from
    FROM tmp2
    GROUP BY 1,2,3,4,5,6
),
tmp4 AS (
    SELECT 
        assortment, article, date_from, date_to, last_update_date, is_delete,vendor_id,
        LEAD(last_update_date) OVER (PARTITION BY assortment, article ORDER BY last_update_date DESC, date_to DESC) prev_update_date,
        LEAD(date_from) OVER (PARTITION BY assortment, article ORDER BY last_update_date DESC, date_to DESC) prev_date_from,
        LEAD(date_to) OVER (PARTITION BY assortment, article ORDER BY last_update_date DESC, date_to DESC) prev_date_to,
        LAG(last_update_date) OVER (PARTITION BY assortment, article ORDER BY last_update_date DESC, date_to DESC) next_update_date,
        LAG(date_from) OVER (PARTITION BY assortment, article ORDER BY last_update_date DESC, date_to DESC) next_date_from,
        LAG(date_to) OVER (PARTITION BY assortment, article ORDER BY last_update_date DESC, date_to DESC) next_date_to
    FROM tmp3
),
tmp5 AS (
    SELECT 
        assortment, article, is_delete, vendor_id, last_update_date,
        IF(date_to IS NULL, NULL, LEAST(date_to,CASE WHEN next_date_from IS NOT NULL THEN DATE_SUB(next_date_from, 1) ELSE date_to END)) date_to,
        date_from
    FROM tmp4
)
SELECT    
    a.assortment,	
    a.article product_id,	
    a.date_from start_date,
    a.date_to end_date,
    a.is_delete,
    a.vendor_id,
    IF(b.product_status LIKE 'Z%', 1, 0) block_by_article,
    IF(c.status = 'ACTIVE', 0, 1) block_by_assortment,
    IF(a.assortment NOT LIKE 'EX_%', 0, 1) block_by_ex,
    a.last_update_date
FROM tmp5 a
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.article = b.product_id
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_desc c ON a.assortment = c.assortment
WHERE date_from <= date_to;
""")